In [ ]:
import nltk
import sklearn as sk
import pandas as pd
import numpy as np
import re

In [ ]:
KB = [
    "∀𝑥:(𝑇(𝑥) => ~𝑀(𝑥))",
    "∀𝑥∀𝑦∀𝑧:(𝑀(𝑥) => ~𝑀(𝑦) ∧ ~𝑀(𝑧))",
    "∀x∀y∀z:(M(x) => (∃u G(u) ∧ H(x, y, z)))",
    "T(𝐴𝑟𝑡ℎ𝑢𝑟) => 𝑀(𝐶𝑎𝑟𝑙𝑒𝑡𝑜𝑛)",
    "~(T(𝐵𝑒𝑟𝑡𝑟𝑎𝑚) ∧ ~~𝑀(𝐵𝑒𝑟𝑡𝑟𝑎𝑚))",
    "T(𝐶𝑎𝑟𝑙𝑒𝑡𝑜𝑛) => ~𝑀(𝐶𝑎𝑟𝑙𝑒𝑡𝑜𝑛)",
    "~~T(𝐴𝑟𝑡ℎ𝑢𝑟)",
    "T(𝐵𝑒𝑟𝑡𝑟𝑎𝑚)"]

goal = ["~𝑇(𝐶𝑎𝑟𝑙𝑒𝑡𝑜𝑛) ", "~𝑀(𝐴𝑟𝑡ℎ𝑢𝑟)"]

#1- Eliminate implication

In [ ]:
def eliminate_implication(KB):
    new_KB = []
    for sentence in KB:
        if "=>" in sentence:
            # Check if the sentence contains a colon indicating a quantifier
            if ":" in sentence:
                quantifier, expression = sentence.split(":", 1)
            else:
                quantifier = ""
                expression = sentence

            # Split the expression into antecedent and consequent parts
            part_before, part_after = expression.split("=>")
            part_before = part_before.strip(" ()")
            part_after = part_after.strip(" ()")

            # Convert the implication to a disjunction
            converted_sentence = f"~{part_before}) ∨ {part_after})"

            # Reassemble the sentence with the quantifier if it exists
            if quantifier.strip():
                new_KB.append(f"{quantifier.strip()}:( {converted_sentence} )")
            else:
                new_KB.append(converted_sentence)
        else:
            # If there is no implication, add the sentence as it is
            new_KB.append(sentence.strip())
    return new_KB


new_KB = eliminate_implication(KB)
print("Converted KB:")
for statement in new_KB:
    print(statement)

Converted KB:
∀𝑥:( ~𝑇(𝑥) ∨ ~𝑀(𝑥) )
∀𝑥∀𝑦∀𝑧:( ~𝑀(𝑥) ∨ ~𝑀(𝑦) ∧ ~𝑀(𝑧) )
∀x∀y∀z:( ~M(x) ∨ ∃u G(u) ∧ H(x, y, z) )
~T(𝐴𝑟𝑡ℎ𝑢𝑟) ∨ 𝑀(𝐶𝑎𝑟𝑙𝑒𝑡𝑜𝑛)
~(T(𝐵𝑒𝑟𝑡𝑟𝑎𝑚) ∧ ~~𝑀(𝐵𝑒𝑟𝑡𝑟𝑎𝑚))
~T(𝐶𝑎𝑟𝑙𝑒𝑡𝑜𝑛) ∨ ~𝑀(𝐶𝑎𝑟𝑙𝑒𝑡𝑜𝑛)
~~T(𝐴𝑟𝑡ℎ𝑢𝑟)
T(𝐵𝑒𝑟𝑡𝑟𝑎𝑚)


# Remove_double_not

In [ ]:
def remove_double_negation(KB):
    for i in range(len(KB)):
        KB[i] = KB[i].replace("~~", "")

# Update new_KB to remove double negations
remove_double_negation(new_KB)

print("Updated KB after removing double negations:")
for statement in new_KB:
    print(statement)


Updated KB after removing double negations:
∀𝑥:( ~𝑇(𝑥) ∨ ~𝑀(𝑥) )
∀𝑥∀𝑦∀𝑧:( ~𝑀(𝑥) ∨ ~𝑀(𝑦) ∧ ~𝑀(𝑧) )
∀x∀y∀z:( ~M(x) ∨ ∃u G(u) ∧ H(x, y, z) )
~T(𝐴𝑟𝑡ℎ𝑢𝑟) ∨ 𝑀(𝐶𝑎𝑟𝑙𝑒𝑡𝑜𝑛)
~(T(𝐵𝑒𝑟𝑡𝑟𝑎𝑚) ∧ 𝑀(𝐵𝑒𝑟𝑡𝑟𝑎𝑚))
~T(𝐶𝑎𝑟𝑙𝑒𝑡𝑜𝑛) ∨ ~𝑀(𝐶𝑎𝑟𝑙𝑒𝑡𝑜𝑛)
T(𝐴𝑟𝑡ℎ𝑢𝑟)
T(𝐵𝑒𝑟𝑡𝑟𝑎𝑚)


# move_negation

In [ ]:
def move_negation(statement):
    if '~(' in statement and '∧' in statement:
        statement = statement.replace('~(', '')
        part_before, part_after = statement.split('∧')
        part_before = f"~{part_before.strip()}"
        part_after = f"~{part_after.strip()}"
        return f"{part_before} ∨ {part_after}"

    elif '~(' in statement and '∨' in statement:
        statement = statement.replace('~(', '').replace(')', '')
        part_before, part_after = statement.split('∨')
        part_before = f"~{part_before.strip()}"
        part_after = f"~{part_after.strip()}"
        return f"{part_before} ∧ {part_after}"
    else:
        return statement


for i in range(len(new_KB)):
    new_KB[i] = move_negation(new_KB[i])

for statement in new_KB:
    print(statement)


∀𝑥:( ~𝑇(𝑥) ∨ ~𝑀(𝑥) )
∀𝑥∀𝑦∀𝑧:( ~𝑀(𝑥) ∨ ~𝑀(𝑦) ∧ ~𝑀(𝑧) )
∀x∀y∀z:( ~M(x) ∨ ∃u G(u) ∧ H(x, y, z) )
~T(𝐴𝑟𝑡ℎ𝑢𝑟) ∨ 𝑀(𝐶𝑎𝑟𝑙𝑒𝑡𝑜𝑛)
~T(𝐵𝑒𝑟𝑡𝑟𝑎𝑚) ∨ ~𝑀(𝐵𝑒𝑟𝑡𝑟𝑎𝑚))
~T(𝐶𝑎𝑟𝑙𝑒𝑡𝑜𝑛) ∨ ~𝑀(𝐶𝑎𝑟𝑙𝑒𝑡𝑜𝑛)
T(𝐴𝑟𝑡ℎ𝑢𝑟)
T(𝐵𝑒𝑟𝑡𝑟𝑎𝑚)


# standardize_variable

In [ ]:

def standardize_variables(KB):
    for i, statement in enumerate(KB):
        variable_map = {}
        new_statement = statement
        quantified_vars = re.findall(r'([∀∃])(\w+)', statement)

        for quantifier, var in quantified_vars:
            if var not in variable_map:
                # Create a unique new variable name within the scope of this statement
                new_var = f"{var}_{len(variable_map)+1}"
                variable_map[var] = new_var
            else:
                new_var = variable_map[var]

            # Replace the old variable with the new one in the statement
            new_statement = re.sub(rf'(?<=[∀∃]){var}\b', new_var, new_statement)
            new_statement = re.sub(rf'\b{var}\b', new_var, new_statement)

        KB[i] = (new_statement)

    return KB
standardized_KB = standardize_variables(new_KB)
print("Knowledge Base after standardizing variables:")
for statement in standardized_KB:
    print(statement)

Knowledge Base after standardizing variables:
∀𝑥_1:( ~𝑇(𝑥_1) ∨ ~𝑀(𝑥_1) )
∀𝑥_1∀𝑦_2∀𝑧_3:( ~𝑀(𝑥_1) ∨ ~𝑀(𝑦_2) ∧ ~𝑀(𝑧_3) )
∀x_1∀y_2∀z_3:( ~M(x_1) ∨ ∃u_4 G(u_4) ∧ H(x_1, y_2, z_3) )
~T(𝐴𝑟𝑡ℎ𝑢𝑟) ∨ 𝑀(𝐶𝑎𝑟𝑙𝑒𝑡𝑜𝑛)
~T(𝐵𝑒𝑟𝑡𝑟𝑎𝑚) ∨ ~𝑀(𝐵𝑒𝑟𝑡𝑟𝑎𝑚))
~T(𝐶𝑎𝑟𝑙𝑒𝑡𝑜𝑛) ∨ ~𝑀(𝐶𝑎𝑟𝑙𝑒𝑡𝑜𝑛)
T(𝐴𝑟𝑡ℎ𝑢𝑟)
T(𝐵𝑒𝑟𝑡𝑟𝑎𝑚)


# move_quantifier_to_left

In [ ]:
def move_quantifiers_to_left(KB):
    for i, statement in enumerate(KB):
        # Extract quantifiers and the rest of the statement
        parts = re.findall(r'([∀∃]\w+)', statement)
        rest = re.sub(r'[∀∃]\w+', '', statement)

        # Concatenate all quantifiers and then append the rest of the statement
        new_statement = ' '.join(parts) + ' ' + rest.strip()
        KB[i] = (new_statement)

    return KB
moved_quantifiers_new_KB = move_quantifiers_to_left(new_KB)

print("Knowledge Base after moving quantifiers to left:")
for statement in moved_quantifiers_new_KB:
    print(statement)

Knowledge Base after moving quantifiers to left:
∀𝑥_1 :( ~𝑇(𝑥_1) ∨ ~𝑀(𝑥_1) )
∀𝑥_1 ∀𝑦_2 ∀𝑧_3 :( ~𝑀(𝑥_1) ∨ ~𝑀(𝑦_2) ∧ ~𝑀(𝑧_3) )
∀x_1 ∀y_2 ∀z_3 ∃u_4 :( ~M(x_1) ∨  G(u_4) ∧ H(x_1, y_2, z_3) )
 ~T(𝐴𝑟𝑡ℎ𝑢𝑟) ∨ 𝑀(𝐶𝑎𝑟𝑙𝑒𝑡𝑜𝑛)
 ~T(𝐵𝑒𝑟𝑡𝑟𝑎𝑚) ∨ ~𝑀(𝐵𝑒𝑟𝑡𝑟𝑎𝑚))
 ~T(𝐶𝑎𝑟𝑙𝑒𝑡𝑜𝑛) ∨ ~𝑀(𝐶𝑎𝑟𝑙𝑒𝑡𝑜𝑛)
 T(𝐴𝑟𝑡ℎ𝑢𝑟)
 T(𝐵𝑒𝑟𝑡𝑟𝑎𝑚)


# Eliminat_existential

In [ ]:
def eliminate_existential(KB):
    for i in range(len(KB)):
        # Use regex to remove ∀ and any following variable
        KB[i] = re.sub(r'∃\w+', '', KB[i])
        KB[i] = re.sub(r'\bu_4\b', 'A', KB[i])

eliminate_existential(new_KB)

for statement in new_KB:
    print(statement)

∀𝑥_1 :( ~𝑇(𝑥_1) ∨ ~𝑀(𝑥_1) )
∀𝑥_1 ∀𝑦_2 ∀𝑧_3 :( ~𝑀(𝑥_1) ∨ ~𝑀(𝑦_2) ∧ ~𝑀(𝑧_3) )
∀x_1 ∀y_2 ∀z_3  :( ~M(x_1) ∨  G(A) ∧ H(x_1, y_2, z_3) )
 ~T(𝐴𝑟𝑡ℎ𝑢𝑟) ∨ 𝑀(𝐶𝑎𝑟𝑙𝑒𝑡𝑜𝑛)
 ~T(𝐵𝑒𝑟𝑡𝑟𝑎𝑚) ∨ ~𝑀(𝐵𝑒𝑟𝑡𝑟𝑎𝑚))
 ~T(𝐶𝑎𝑟𝑙𝑒𝑡𝑜𝑛) ∨ ~𝑀(𝐶𝑎𝑟𝑙𝑒𝑡𝑜𝑛)
 T(𝐴𝑟𝑡ℎ𝑢𝑟)
 T(𝐵𝑒𝑟𝑡𝑟𝑎𝑚)


# Eliminate_Universal

In [ ]:
def eliminate_universal(KB):
    for i in range(len(KB)):
        # Use regex to remove ∀ and any following variable, including optional whitespace
        KB[i] = re.sub(r'∀\w+\s?', '', KB[i])

eliminate_universal(new_KB)

for statement in new_KB:
    print(statement)


:( ~𝑇(𝑥_1) ∨ ~𝑀(𝑥_1) )
:( ~𝑀(𝑥_1) ∨ ~𝑀(𝑦_2) ∧ ~𝑀(𝑧_3) )
 :( ~M(x_1) ∨  G(A) ∧ H(x_1, y_2, z_3) )
 ~T(𝐴𝑟𝑡ℎ𝑢𝑟) ∨ 𝑀(𝐶𝑎𝑟𝑙𝑒𝑡𝑜𝑛)
 ~T(𝐵𝑒𝑟𝑡𝑟𝑎𝑚) ∨ ~𝑀(𝐵𝑒𝑟𝑡𝑟𝑎𝑚))
 ~T(𝐶𝑎𝑟𝑙𝑒𝑡𝑜𝑛) ∨ ~𝑀(𝐶𝑎𝑟𝑙𝑒𝑡𝑜𝑛)
 T(𝐴𝑟𝑡ℎ𝑢𝑟)
 T(𝐵𝑒𝑟𝑡𝑟𝑎𝑚)


# Distribution

In [ ]:
def distribution(statement):
    if '∧' in statement:
        parts = re.findall(r'(\w+\([^)]+\))', statement)
        if len(parts) > 2:
            first_part = parts[0]
            remaining_parts = parts[1:]
            distributed_parts = []

            for part in remaining_parts:
                distributed_part = f"{first_part} ∨ {part}"
                distributed_parts.append(distributed_part)

            return '\n'.join(distributed_parts)
    return statement

for i in range(len(new_KB)):
    new_KB[i] = distribution(new_KB[i])

for statement in new_KB:
    print(statement)

:( ~𝑇(𝑥_1) ∨ ~𝑀(𝑥_1) )
𝑀(𝑥_1) ∨ 𝑀(𝑦_2)
𝑀(𝑥_1) ∨ 𝑀(𝑧_3)
M(x_1) ∨ G(A)
M(x_1) ∨ H(x_1, y_2, z_3)
 ~T(𝐴𝑟𝑡ℎ𝑢𝑟) ∨ 𝑀(𝐶𝑎𝑟𝑙𝑒𝑡𝑜𝑛)
 ~T(𝐵𝑒𝑟𝑡𝑟𝑎𝑚) ∨ ~𝑀(𝐵𝑒𝑟𝑡𝑟𝑎𝑚))
 ~T(𝐶𝑎𝑟𝑙𝑒𝑡𝑜𝑛) ∨ ~𝑀(𝐶𝑎𝑟𝑙𝑒𝑡𝑜𝑛)
 T(𝐴𝑟𝑡ℎ𝑢𝑟)
 T(𝐵𝑒𝑟𝑡𝑟𝑎𝑚)


# Negate_goal

In [ ]:
for statement in goal:
    negated_goal = "~" + statement.strip("~")
    new_KB.append(negated_goal)

for statement in new_KB:
    print(statement)

:( ~𝑇(𝑥_1) ∨ ~𝑀(𝑥_1) )
𝑀(𝑥_1) ∨ 𝑀(𝑦_2)
𝑀(𝑥_1) ∨ 𝑀(𝑧_3)
M(x_1) ∨ G(A)
M(x_1) ∨ H(x_1, y_2, z_3)
 ~T(𝐴𝑟𝑡ℎ𝑢𝑟) ∨ 𝑀(𝐶𝑎𝑟𝑙𝑒𝑡𝑜𝑛)
 ~T(𝐵𝑒𝑟𝑡𝑟𝑎𝑚) ∨ ~𝑀(𝐵𝑒𝑟𝑡𝑟𝑎𝑚))
 ~T(𝐶𝑎𝑟𝑙𝑒𝑡𝑜𝑛) ∨ ~𝑀(𝐶𝑎𝑟𝑙𝑒𝑡𝑜𝑛)
 T(𝐴𝑟𝑡ℎ𝑢𝑟)
 T(𝐵𝑒𝑟𝑡𝑟𝑎𝑚)
~𝑇(𝐶𝑎𝑟𝑙𝑒𝑡𝑜𝑛) 
~𝑀(𝐴𝑟𝑡ℎ𝑢𝑟)


# Arrive to empty clause{ }

In [ ]:
def resolution(KB):
    # Normalize the kb into CNF if not already in CNF
    normalized_KB = KB

    # Initialize a set or list to hold new clauses derived during the process
    new_clauses = set()

    while True:
        # Apply the resolution rule to all pairs of clauses in the kb
        for i in range(len(normalized_KB)):
            for j in range(i + 1, len(normalized_KB)):
                resolved_clause = resolve(normalized_KB[i], normalized_KB[j])

                # Check if the resolved_clause is the empty
                if resolved_clause == "{}":
                    return True  # Empty clause found, indicating a contradiction or proof completion

                if resolved_clause and resolved_clause not in normalized_KB:
                    new_clauses.add(resolved_clause)

        # If no new clauses were added, stop the process
        if not new_clauses:
            break

        # Add the newly derived clauses to the kb
        normalized_KB.update(new_clauses)
        new_clauses.clear()

    return False

def resolve(clause1, clause2):
    # Split the clauses into sets of literals
    literals1 = set(clause1.split(' ∨ '))
    literals2 = set(clause2.split(' ∨ '))

    # Find complementary literals and resolve
    for literal in literals1:
        negation = '¬' + literal if literal[0] != '¬' else literal[1:]
        if negation in literals2:
            # Remove the complementary literals
            new_literals1 = literals1 - {literal}
            new_literals2 = literals2 - {negation}
            # Combine the remaining literals to form the resolved clause
            resolved_clause = ' ∨ '.join(new_literals1 | new_literals2)
            return resolved_clause if resolved_clause else "{}"  # Return empty clause if no literals remain

    # Return None if no resolution is possible
    return None


# Assuming new_KB is your knowledge base
result = resolution(new_KB)
print("Empty clause derived:", result)


Empty clause derived: False


# Another way to prove an empty clause

In [ ]:
new_KB = """
~T(x_1) ∨ ~M(x_1)
~M(x_1) ∨ ~M(y_2)
~M(x_1) ∨ ~M(z_3)
~M(x_1) ∨ G(A)
~M(x_1) ∨ H(x_1, y_2, z_3)
~T(Arthur) ∨ M(Carleton)
~T(Bertram) ∨ ~M(Bertram)
~T(Carleton) ∨ ~M(Carleton)
T(Arthur)
T(Bertram)
~T(Carleton)
~M(Arthur)
"""


# Splitting the KB into individual clauses
clauses = new_KB.strip().split('\n')

# Splitting each clause at '∨' and adding to the set, while removing content within parentheses
terms_set = set()
for clause in clauses:
    terms = clause.split('∨')
    for term in terms:
        # Removing content within parentheses
        cleaned_term = re.sub(r'\(.*?\)', '', term.strip())
        terms_set.add(cleaned_term)


terms_set

{'G', 'H', 'M', 'T', '~M', '~T'}

In [ ]:
#if sumition of resolution is 0 that mean it consistent else it will be Non_consistent
sum = 0
for i in terms_set:
    if i.startswith('~'):
        sum -= 1
    else:
        sum += 1
if sum==0:
    print("the resolution is consistent and the result of sum the list is : ", sum)
else:
    print("the resolution is Non_consistent and the result of sum the list is :", sum)


the resolution is Non_consistent and the result of sum the list is : 2
